In [ ]:
from dask.distributed import Client as DaskClient
from dep_tools.grids import PACIFIC_GRID_10
from dep_tools.loaders import OdcLoader
from dep_tools.searchers import PystacSearcher
from planetary_computer import sign_url

from src.utils import GeoMADSentinel1Processor

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Study site configuration
# item_id = "63,20"  # West viti levu
# item_id = "65,22"  # Near the anti-meridian
# item_id = "66,22"  # Right of the anti-meridian
# item_id = "48,16"  # Vanuatu, middle of Grand Terre
# item_id = "89,16"  # Rarotonga

# Failing
# item_id = "42,13"  # No vv band, only hh and hv
item_id = "66,22"  # Anti-meridian area with missing data

datetime = "2024"

# And get the study site
tile_index = tuple(int(i) for i in item_id.split(","))
geobox = PACIFIC_GRID_10.tile_geobox(tile_index)

# Load low-res if you want it faster
geobox = geobox.zoom_out(10)

geobox.explore()

In [ ]:
# Only use descending data
query = {
    "sat:orbit_state": {"eq": "descending"},
    "sar:polarizations": {"eq": ["VV", "VH"]},  # Use only VV and VH polarizations
}
search_kwargs = {"query": query}

# Search for data
searcher = PystacSearcher(
    catalog="https://planetarycomputer.microsoft.com/api/stac/v1/",
    collections=["sentinel-1-rtc"],
    datetime=datetime,
    **search_kwargs
)

items = searcher.search(area=geobox)
print(f"Found {len(items)} items")

In [ ]:
from dep_tools.utils import bbox_across_180

bbox = bbox_across_180(geobox)
bbox

In [ ]:
import geopandas as gpd
from shapely.geometry import shape
from dep_tools.utils import bbox_across_180

bbox = bbox_across_180(geobox)
bbox

# Convert the stac geometries to geopandas and plot
geoms = []

for item in items:
    geom = item.geometry
    geoms.append(geom)


gdf = gpd.GeoDataFrame(geometry=[shape(geom) for geom in geoms], crs="EPSG:4326")
gdf.explore()

In [ ]:
item.geometry

In [ ]:
item.self_href

In [ ]:
item.geometry


In [ ]:
import geopandas as gpd
from shapely.geometry import shape

# Convert the stac geometries to geopandas and plot
geoms = []

for item in items:
    geom = item.geometry
    geoms.append(geom)


gdf = gpd.GeoDataFrame(geometry=[shape(geom) for geom in geoms], crs="EPSG:4326")
gdf.explore()

In [ ]:
load_kwargs = {"patch_url": sign_url}

# Set up a data loader
loader = OdcLoader(
    bands=["vv", "vh"],
    chunks=dict(time=1, x=3201, y=3201),
    groupby="solar_day",
    fail_on_error=False,
    **load_kwargs
)

# Run the load process, which is lazy-loaded
input_data = loader.load(items, areas=geobox)
input_data

In [ ]:
# Set up a data processor
processor = GeoMADSentinel1Processor(
    geomad_options=dict(
        work_chunks=(600, 600),
        num_threads=10,
        maxiters=100,
    ),
    min_timesteps=5,
)

# Do the processing using a local dask cluster
with DaskClient(n_workers=1, threads_per_worker=32, memory_limit="20GB"):
    output_data = processor.process(input_data)

output_data

In [ ]:
import folium
from ipyleaflet import basemaps

m = folium.Map(
    location=geobox.geographic_extent.centroid.coords[0][::-1],
    zoom_start=8,
    tiles=basemaps.Esri.WorldImagery,
)

for band in output_data.data_vars:
    output_data[band].odc.add_to(m)

m.add_child(folium.LayerControl())

m

In [ ]:
output_data.mean_vv.plot.imshow()

In [ ]:
from pystac import Item
from odc.stac import load

item = Item.from_file("https://dep-public-staging.s3.us-west-2.amazonaws.com/dep_s1_geomad/0-1-2/071/017/2024/dep_s1_geomad_071_017_2024.stac-item.json")

data = load([item], chunks={})
data

In [ ]:
data.vv.odc.explore()